In [1]:
import pandas as pd
import json
import os
from sklearn.model_selection import train_test_split

# ================= 配置 =================
INPUT_CSV = "gold_training_data_scored.csv"  # 你之前生成的带分数的CSV
TRAIN_FILE = "gold_llm_train.jsonl"
TEST_FILE = "gold_llm_test.jsonl"

# 严格按时间切分，防止未来数据泄露
# 假设数据是 2015-2025，我们用 2025 年做测试
TEST_START_DATE = "2025-06-01"

def format_data():
    print(f"📂 读取数据: {INPUT_CSV} ...")
    if not os.path.exists(INPUT_CSV):
        print("❌ 错误：找不到CSV文件，请确保你已经完成了数据收集和打分步骤。")
        return

    df = pd.read_csv(INPUT_CSV)
    df['Date'] = pd.to_datetime(df['Date'])
    
    # 过滤掉分数缺失的数据
    df = df.dropna(subset=['Score', 'Headline'])
    
    # 1. 时间切分
    train_df = df[df['Date'] < TEST_START_DATE].copy()
    test_df = df[df['Date'] >= TEST_START_DATE].copy()
    
    print(f"   训练集 (2020-2025): {len(train_df)} 条")
    print(f"   测试集 (2025):      {len(test_df)} 条")

    # 2. 定义 Prompt 模板
    # 我们教 AI 输出 JSON 格式，方便后续提取分数
    system_instruction = """You are a Macro Quant Strategist specializing in Gold (XAU/USD).
Analyze the given news headline and assign a Sentiment Score from -5 to +5.
Scoring Rules:
-5: Extreme Bearish (e.g., Unexpected Rate Hike, War Ends)
-1 to -4: Bearish (e.g., Strong Dollar, Inflation Cooling)
 0: Neutral / Noise
+1 to +4: Bullish (e.g., Rate Cut Hints, Central Bank Buying)
+5: Extreme Bullish (e.g., War Outbreak, Financial Crisis)
"""

    def create_jsonl(dataframe, filename):
        with open(filename, 'w', encoding='utf-8') as f:
            for _, row in dataframe.iterrows():
                score = int(row['Score'])
                
                # 简单的逻辑生成 (CoT) - 模拟专家的口吻
                sentiment_str = "Bullish" if score > 0 else ("Bearish" if score < 0 else "Neutral")
                

                input_text = f"Date: {row['Date'].strftime('%Y-%m-%d')}\nNews: {row['Headline']}"
                
                output_text = f"""Score: {score}"""

                entry = {
                    "instruction": system_instruction,
                    "input": input_text,
                    "output": output_text
                }
                
                json.dump(entry, f)
                f.write('\n')
        print(f"💾 已保存: {filename}")

    create_jsonl(train_df, TRAIN_FILE)
    create_jsonl(test_df, TEST_FILE)
    print("✅ 格式化完成！准备进入训练阶段。")

if __name__ == "__main__":
    format_data()

📂 读取数据: gold_training_data_scored.csv ...
   训练集 (2020-2025): 14416 条
   测试集 (2025):      1880 条
💾 已保存: gold_llm_train.jsonl
💾 已保存: gold_llm_test.jsonl
✅ 格式化完成！准备进入训练阶段。
